### Задание 1**
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [18]:
import pandas as pd
df = pd.read_csv('ratings.csv')

def class_ratings(row):
    if row['rating'] <= 2:
        rating = 'низкий'
    elif 2 < row['rating'] <= 4:
        rating = 'средний'
    elif 4 < row['rating'] <= 5:
        rating = 'высокий'
    return rating


df['class_ratings'] = df.apply(class_ratings, axis = 1)
df.head()

,userId,movieId,rating,timestamp,class_ratings
0,1,31,2.5,1260759144,средний
1,1,1029,3.0,1260759179,средний
2,1,1061,3.0,1260759182,средний
3,1,1129,2.0,1260759185,низкий
4,1,1172,4.0,1260759205,средний


### Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

12345
'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

Результат классификации запишите в отдельный столбец region.

In [28]:
import pandas as pd


df = pd.read_csv('keywords.csv')
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}


def region(row):
    for k, v in geo_data.items():
        if row['keyword'] in v:
            return k
    return 'undefined'


df['region'] = df.apply(region, axis = 1)
df.groupby('region').count()

,keyword,shows
region,,
undefined,99991,99991
Дальний Восток,3,3
Северо-Запад,3,3
Центр,3,3


### Задание 3 (бонусное)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

    В переменную years запишите список из всех годов с 1950 по 2010.

    Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
        для каждой строки пройдите по всем годам списка years
        если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
        если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

    Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

    Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

P/S/ все файлы находятся в архиве «дополнительные файлы к лекции и дз»

In [77]:
import pandas as pd
import re

movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

"""
А так можно было сделать?
Мы тогда будем не ограничены задаваемым далее периодом (а в файле есть фильмы с 1902 по 2016),
да и лист не надо будет формировать. И работает быстрее.
"""
def production_year(row):
    year = re.findall(r'\d{4}', row['title'])
    if len(year) > 0:
        return int(year[len(year)-1]) # чтоб последний смотреть, если само название фильма содержит год
    return 1900

"""
years = []
for i in range(1950, 2011):
    years.append(i)

def production_year(row):
    for i in years:
        if str(i) in row['title']:
            return i
    return 1900
"""
movies['years'] = movies.apply(production_year, axis = 1)
# movies.sort_values('years', ascending=True).head()
df = movies.merge(ratings[['movieId','rating']], on='movieId')[['years', 'rating']]

df.groupby(['years']).mean().sort_values('rating', ascending=False)

,rating
years,
1921,4.416667
1902,4.333333
1928,4.261905
1917,4.250000
1918,4.250000
...,...
2013,3.356973
2000,3.355945
2016,3.217742
